In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json

In [3]:
# Directory where the Excel files are stored
base_path = "/Users/tinazhang/Downloads/"

# state data
years = list(range(2001, 2024))  # From 2001 to 2023
states_data = {}


for year in years:
    file_name = f"state_M{year}_dl.csv"
    file_path = base_path + file_name

    # get dataframe
    df = pd.read_csv(file_path)
    try:
        data_filtered = df[['OCC_CODE', 'TOT_EMP', 'A_MEAN', 'A_MEDIAN', 'AREA_TITLE']]
        states_data[year] = data_filtered
    # if other key names
    except KeyError:
        try:
            try:
                data_filtered = df[['occ_code', 'tot_emp', 'a_mean', 'a_median', 'area_title']]
            except:
                data_filtered = df[['OCC_CODE', 'TOT_EMP', 'A_MEAN', 'A_MEDIAN','STATE']]
        except:
            data_filtered = df[['occ_code', 'tot_emp', 'a_mean', 'a_median','state']]

    # rename cols
    data_filtered.columns = ["OCC_CODE", "TOT_EMP", "A_MEAN", "A_MEDIAN", "AREA"]

    states_data[year] = data_filtered

In [4]:
# map 2010 SOC codes to 2018 SOC codes
crosswalk1 = pd.read_excel("/Users/tinazhang/Downloads/soc_2010_to_2018_crosswalk.xlsx")
soc_mapping1 = dict(zip(crosswalk1['U.S. Bureau of Labor Statistics'], crosswalk1['Unnamed: 2']))

In [5]:
# map 2000 SOC codes to 2010 SOC codes
crosswalk2 = pd.read_excel("/Users/tinazhang/Downloads/soc_2000_to_2010_crosswalk.xlsx")
soc_mapping2 = dict(zip(crosswalk2['Bureau of Labor Statistics'], crosswalk2['Unnamed: 2']))

In [6]:
# remove all columns except OCC_CODE, A_MEAN, A_MEDIAN, TOT_EMP
for year, data in states_data.items():
    data_copy = data.copy()
    
    data_copy["TOT_EMP"]=data_copy["TOT_EMP"].astype(str).str.replace(",", "")
    data_copy["TOT_EMP"]=data_copy["TOT_EMP"].str.replace("*", "")
    data_copy["A_MEAN"]=data_copy["A_MEAN"].astype(str).str.replace(",", "")
    data_copy["A_MEAN"]=data_copy["A_MEAN"].str.replace("*", "")
    data_copy["A_MEDIAN"]=data_copy["A_MEDIAN"].astype(str).str.replace(",", "")
    data_copy["A_MEDIAN"]=data_copy["A_MEDIAN"].str.replace("*", "")

    data_copy["TOT_EMP"]=pd.to_numeric(data_copy["TOT_EMP"], errors="coerce").astype(float)
    data_copy["A_MEAN"]=pd.to_numeric(data_copy["A_MEAN"], errors="coerce").astype(float)
    data_copy["A_MEDIAN"]=pd.to_numeric(data_copy["A_MEDIAN"], errors="coerce").astype(float)
    data_copy["YEAR"] = year

    data_copy.rename(columns = {"OCC_CODE": "original_soc"}, inplace=True)
    data_copy['2018_soc'] = data_copy["original_soc"].map(soc_mapping1)
    data_copy['2010_soc'] = data_copy["original_soc"].map(soc_mapping2)
    
    states_data[year] = data_copy

In [7]:
# USE THIS TO CHECK presence of certain data

url = "https://apps.bea.gov/api/data"
params = {
    "UserID": "FF6C632C-0E8F-4A78-AA0C-81B00FDB0113",
    "method": "GETPARAMETERVALUESFILTERED",
    "DatasetName": "REGIONAL",
    "GeoFIPS": "STATE",
    "LineCode": "30",
    "TableName": "SAINC4",
    "Year": "2000",  
    "ResultFormat": "JSON",
    "TargetParameter": "LINECODE"
}

response = requests.get(url, params=params)

# Check if the response was successful
if response.status_code == 200:
    # Parse and print the JSON response
    data = response.json()

    # Find the relevant data for 'Personal income' (Key 10)
    personal_income_data = None
    for param in data['BEAAPI']['Results']['ParamValue']:
        if param['Key'] == "30":  # Key 10 corresponds to Personal income
            personal_income_data = param
            break

    if personal_income_data:
        print("Personal income data found:")
        print(json.dumps(personal_income_data, indent=4))
    else:
        print("Personal income data not found.")
else:
    print(f"Error")

Personal income data found:
{
    "Key": "30",
    "Desc": "[SAINC4] Per capita personal income"
}


In [8]:
def fetch_data(url, params, start_year=2023, end_year=2001):
    """
    Fetches data for a range of years from the BEA API.

    Parameters:
    - url (str): The BEA API endpoint URL
    - params (dict): A dictionary of parameters for the API request (excluding 'Year')
    - start_year (int): The first year to fetch data for (default: 2023)
    - end_year (int): The last year to fetch data for (default: 2001)

    Returns:
    - pandas DataFrame: DataFrame containing the requested data
    """
    
    # this is to hold data
    all_data = []

    # Loop over years from start_year to end_year (inclusive)
    for year in range(start_year, end_year - 1, -1):
        params["Year"] = str(year)  

        response = requests.get(url, params=params)
        
        # if response successful
        if response.status_code == 200:
            data = response.json()
            
            # check for expected data
            if 'BEAAPI' in data and 'Results' in data['BEAAPI'] and 'Data' in data['BEAAPI']['Results']:
                data_rows = data['BEAAPI']['Results']['Data']
                all_data.extend(data_rows)
            else:
                print(f"No data found for {year}.")
        else:
            print(f"Error fetching data for {year}. Status code: {response.status_code}")


    if all_data:
        df = pd.DataFrame(all_data)
        df = df[['GeoFips', 'GeoName', 'TimePeriod', 'DataValue']]
        return df
    else:
        print("No data retrieved.")
        return pd.DataFrame()

In [9]:
# PERSONAL INCOME, POPULATION, INCOME PER CAPITA

# PERSONAL INCOME
url = "https://apps.bea.gov/api/data"
income_params = {
    "UserID": "FF6C632C-0E8F-4A78-AA0C-81B00FDB0113",
    "method": "GETDATA",
    "DatasetName": "REGIONAL",
    "GeoFIPS": "STATE", 
    "LineCode": "10", 
    "TableName": "SAINC4", 
    "ResultFormat": "JSON", 
}
personal_income = fetch_data(url, income_params)

# POPULATION
population_params = {
    "UserID": "FF6C632C-0E8F-4A78-AA0C-81B00FDB0113",
    "method": "GETDATA",
    "DatasetName": "REGIONAL",
    "GeoFIPS": "STATE", 
    "LineCode": "20", 
    "TableName": "SAINC4", 
    "Year": "2023",
    "ResultFormat": "JSON", 
}
population = fetch_data(url, population_params)

# INCOME PER CAPITA
per_params = {
    "UserID": "FF6C632C-0E8F-4A78-AA0C-81B00FDB0113",
    "method": "GETDATA",
    "DatasetName": "REGIONAL",
    "GeoFIPS": "STATE", 
    "LineCode": "30", 
    "TableName": "SAINC4", 
    "Year": "2023",
    "ResultFormat": "JSON", 
}
income_capita = fetch_data(url, per_params)

# MERGE
# first merge personal income and population
merged_one = pd.merge(personal_income, population, on='GeoName', how='outer')
# merge with income per capita
merged_all = pd.merge(merged_one, income_capita, on='GeoName', how='outer')

merged_all = merged_all[['GeoFips_x', 'GeoName', 'TimePeriod_x', 'DataValue_x', 'DataValue_y', 'DataValue']]
merged_all.rename(columns={'GeoFips_x': 'FIPS_CODE', 'GeoName': 'AREA','TimePeriod_x': 'YEAR', 'DataValue_x': 'PI', 'DataValue_y': 'POPULATION', 'DataValue': 'PIPC'}, inplace=True)

In [10]:
state_to_fips = {
    "Alabama": "01",
    "Alaska": "02",
    "Arizona": "04",
    "Arkansas": "05",
    "California": "06",
    "Colorado": "08",
    "Connecticut": "09",
    "Delaware": "10",
    "Florida": "12",
    "Georgia": "13",
    "Hawaii": "15",
    "Idaho": "16",
    "Illinois": "17",
    "Indiana": "18",
    "Iowa": "19",
    "Kansas": "20",
    "Kentucky": "21",
    "Louisiana": "22",
    "Maine": "23",
    "Maryland": "24",
    "Massachusetts": "25",
    "Michigan": "26",
    "Minnesota": "27",
    "Mississippi": "28",
    "Missouri": "29",
    "Montana": "30",
    "Nebraska": "31",
    "Nevada": "32",
    "New Hampshire": "33",
    "New Jersey": "34",
    "New Mexico": "35",
    "New York": "36",
    "North Carolina": "37",
    "North Dakota": "38",
    "Ohio": "39",
    "Oklahoma": "40",
    "Oregon": "41",
    "Pennsylvania": "42",
    "Rhode Island": "44",
    "South Carolina": "45",
    "South Dakota": "46",
    "Tennessee": "47",
    "Texas": "48",
    "Utah": "49",
    "Vermont": "50",
    "Virginia": "51",
    "Washington": "53",
    "West Virginia": "54",
    "Wisconsin": "55",
    "Wyoming": "56"
}

In [12]:
# We want 
final_data = []


for year, df in states_data.items():
    df['FIPS_CODE'] = df['AREA'].map(state_to_fips)
    
    # Select columns
    df_filtered = df[['AREA', 'FIPS_CODE', 'TOT_EMP', 'original_soc', 'A_MEDIAN', 'A_MEAN', 'YEAR']]
    
    # Append the result to final_data
    final_data.append(df_filtered)


# MERGING
final_df = pd.concat(final_data, ignore_index=True)
final_df['YEAR'] = final_df['YEAR'].astype(str)

final_df[['AREA', 'YEAR']] = final_df[['AREA', 'YEAR']].apply(lambda x: x.str.strip())
merged_all[['AREA', 'YEAR']] = merged_all[['AREA', 'YEAR']].apply(lambda x: x.str.strip())
final_df['FIPS_CODE'] = df['FIPS_CODE'] + "000"

# dictionary mapping from merged_all
mapping = {(row['AREA'], row['YEAR']): row for _, row in merged_all.iterrows()}

# add new columns to final_df by mapping using the dictionary
def map_row(row, col_name):
    key = (row['AREA'], row['YEAR'])
    return mapping.get(key, {}).get(col_name, None)

final_df['PI'] = final_df.apply(lambda row: map_row(row, 'PI'), axis=1)
final_df['POPULATION'] = final_df.apply(lambda row: map_row(row, 'POPULATION'), axis=1)
final_df['PIPC'] = final_df.apply(lambda row: map_row(row, 'PIPC'), axis=1)

print(final_df)

final_df.to_csv('/Users/tinazhang/Downloads/output_file.csv', index=False)

                  AREA FIPS_CODE    TOT_EMP original_soc  A_MEDIAN    A_MEAN  \
0              Alabama     01000  1827960.0      00-0000   22910.0   29530.0   
1              Alabama     01000   107460.0      11-0000   55000.0   61000.0   
2              Alabama     01000     5290.0      11-1011  111740.0  103350.0   
3              Alabama     01000    33420.0      11-1021   55660.0   64170.0   
4              Alabama     01000     1740.0      11-1031   13400.0   15760.0   
...                ...       ...        ...          ...       ...       ...   
836630  Virgin Islands       NaN       30.0      53-7051   37950.0   37650.0   
836631  Virgin Islands       NaN       50.0      53-7061   30460.0   31980.0   
836632  Virgin Islands       NaN      410.0      53-7062   32300.0   33230.0   
836633  Virgin Islands       NaN      100.0      53-7064   24040.0   27410.0   
836634  Virgin Islands       NaN      540.0      53-7065   27290.0   27710.0   

        YEAR        PI POPULATION   PIP